In [ ]:
#using a pretrained cnn - VGG16 (trained on imagenet data set of animals) 
#this is available as an application in keras
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',                          #this argument is weight initialisation we can use either the imagenet weights or random initialisation
            include_top=False,                                 #False => not to use the dense layers 
            input_shape=(150, 150, 3))

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

#defining direcory location for the image input to the network, no need to use os.mkdir here since directory is already made before
base_dir = '/home/anish/Documents/Jupyter Notebook/SA-DLwithPy/CatsVsDogs/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


In [ ]:
#designing the network architecture with the conv base as first layer and then adding dense classifier to it
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#freezing the conv base weights so that it is not modified while feeding the data through it
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))


In [ ]:
#training the model end to end with the frozen conv base and augmented data i/p to the network
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

#augmenting data using imagedatagenerator class of keras
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest')
#validation data is not augmented only rescaled to values btwn 0 and 1 for easy processing of the data
test_datagen = ImageDataGenerator(rescale=1./255)

#generator fot train data
train_generator = train_datagen.flow_from_directory(            #using flow from directory method of imagedatagenerator class to generate datas
                  train_dir,                                    #specifying directory from which data is to be loaded
                  target_size=(150, 150),                       #resizing the image
                  batch_size=20,
                  class_mode='binary')

#generator fot validation data
validation_generator = test_datagen.flow_from_directory(
                       validation_dir,
                       target_size=(150, 150),
                       batch_size=20,
                       class_mode='binary')

#compiling the designed model
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
             metrics=['acc'])

#fitting the generated augmented images to the corresponding labels by using the compiled model 
history = model.fit_generator( 
          train_generator,                                #generating the train data
          steps_per_epoch=100,                            #breaking the data generation and now its time for weight updates, this marks one epoch
          epochs=30,
          validation_data=validation_generator,
          validation_steps=50)


In [ ]:
#plotting the results - training (accuracy and losses) vs validation (accuracy and losses)
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#finetuning - training by unfreezing only few top layers of conv base
conv_base.trainable = True                                #unfreezing the whole convnet

set_trainable = False                                    #initialising a boolean variable

for layer in conv_base.layers:                           #iterating over layers in conv net
    if layer.name == 'block5_conv1':                     #if layer name is block5 then making those layers trainable
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False     
        
        

In [ ]:
#compiling the model with a very low learning rate : so that the trainable weights
#of the convnet is not modified too much, which might harm the representations it is having
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),     #learning rate is set low        
              metrics=['acc'])

#fitting the model with generated augmented i/p images and its labels using the 
#model with top few conv layer set to trainable
history = model.fit_generator(
          train_generator,                              #loading train data from this directory
          steps_per_epoch=100,                          #breaking the data generation, with this one epoch is completed and the trainable weights are updated
          epochs=100,
          validation_data=validation_generator,         #loading the unaugmented validation data
          validation_steps=50)




In [ ]:
#plotting the results after unfreezing top 3 layers of pretrained convnet - training (accuracy and losses) vs validation (accuracy and losses)
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#smoothening the curves
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

plt.plot(epochs,smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs,smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


In [ ]:
#evaluating the model for the test data
#generating test data
test_generator = test_datagen.flow_from_directory(
                 test_dir,
                 target_size=(150, 150),
                 batch_size=20,
                 class_mode='binary')
#evaluating the result of the test data on the trained model
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)

print('test acc:', test_acc)